In [1]:
import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

import pickle

In [2]:
data_folder = os.path.join('..','data')
jan_file = os.path.join(data_folder, 'green_tripdata_2021-01.parquet')
feb_file = os.path.join(data_folder, 'green_tripdata_2021-02.parquet')

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    # Preprocessing and filtering
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds() / 60)
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

    # Feature engineering
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [3]:
df_train = read_dataframe(jan_file)
df_val = read_dataframe(feb_file)

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [4]:
len(df_train), len(df_val)

(73908, 61921)

In [5]:
# Feature engineering
categorical = ['PU_DO']#['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False) # 9 minutes

7.4793657829373155

In [ ]:
# Not really good score
# lr = Lasso(alpha = 0.001)
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared = False) # 9 minutes

In [6]:
# Model dump
models_folder = os.path.join('..','models')
lr_model_file = os.path.join(models_folder, 'lin_reg.bin')
with open(lr_model_file, 'wb') as f_out:
    pickle.dump((dv, lr), f_out)